# YOLOv11s Fine-tuning for Chest X-ray Abnormality Detection


## Section 1: Setup and Imports

In [1]:
# Set working directory to repository root
%cd /home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray

/home/minhquana/workspace/project_DeepLearning/computer_vision/Abnormal-Prediction-In-Chest-X-Ray


In [2]:
# Import required libraries
import os
import shutil
from pathlib import Path
import yaml

import torch
import wandb
from ultralytics import YOLO, settings

# Import custom augmentation
import sys
sys.path.insert(0, str(Path.cwd()))

print("✓ Imports successful")
print(f"  PyTorch version: {torch.__version__}")
print(f"  CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

✓ Imports successful
  PyTorch version: 2.9.0+cu128
  CUDA available: True
  GPU: NVIDIA GeForce RTX 3060
  GPU Memory: 12.5 GB


## Section 2: Verify Preprocessed Data

In [3]:
# Verify preprocessed data directory
preprocessed_dir = Path('data/preprocessed_with_aug')
data_yaml = preprocessed_dir / 'data.yaml'

print("Verifying Preprocessed Data")
print("=" * 80)

if not preprocessed_dir.exists():
    print("ERROR: Preprocessed data not found!")
    print(f"   Expected location: {preprocessed_dir.absolute()}")
    print("\nPlease run data_preparation.ipynb first to create preprocessed data.")
    raise FileNotFoundError(f"Preprocessed data not found at {preprocessed_dir}")

if not data_yaml.exists():
    print(f"ERROR: data.yaml not found at {data_yaml}")
    raise FileNotFoundError(f"data.yaml not found")

print(f"✓ Preprocessed data directory found: {preprocessed_dir}")
print(f"✓ Data YAML found: {data_yaml}")

# Load data.yaml
with open(data_yaml, 'r') as f:
    data_config = yaml.safe_load(f)

print(f"\nDataset Configuration:")
print(f"  Number of classes: {data_config['nc']}")
print(f"  Class names: {data_config['names']}")

# Count images in each split
splits = ['train', 'valid', 'test']
split_counts = {}

for split in splits:
    images_dir = preprocessed_dir / split / 'images'
    if images_dir.exists():
        count = len(list(images_dir.glob('*.png'))) + len(list(images_dir.glob('*.jpg')))
        split_counts[split] = count
    else:
        split_counts[split] = 0

print(f"\nDataset Statistics:")
print(f"  Train:      {split_counts['train']:,} images")
print(f"  Validation: {split_counts['valid']:,} images")
print(f"  Test:       {split_counts['test']:,} images")
print(f"  Total:      {sum(split_counts.values()):,} images")

if split_counts['train'] == 0:
    print("\nERROR: No training images found!")
    raise ValueError("No training images found in preprocessed data")

print("\n✓ Data verification complete - ready for training!")
print("=" * 80)

Verifying Preprocessed Data
✓ Preprocessed data directory found: data/preprocessed_with_aug
✓ Data YAML found: data/preprocessed_with_aug/data.yaml

Dataset Configuration:
  Number of classes: 8
  Class names: ['Aortic enlargement', 'Cardiomegaly', 'Lung Opacity', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pulmonary fibrosis', 'Normal']

Dataset Statistics:
  Train:      10,038 images
  Validation: 1,530 images
  Test:       745 images
  Total:      12,313 images

✓ Data verification complete - ready for training!


## Section 3: WandB Setup

In [4]:
# Login to WandB
wandb.login(key=os.getenv('WANDB_API_KEY'))
print("✓ Logged into Weights & Biases successfully")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/minhquana/.netrc
wandb: Currently logged in as: minhquana (minhquana-university-of-transportation-and-communication) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✓ Logged into Weights & Biases successfully


In [5]:
# Initialize WandB project
wandb.init(
    project="chest-xray-abnormality-detection",
    name="yolov11s-gaussian-blur-rotation",
    config={
        "model": "YOLOv11s",
        "dataset": "VinBigData Chest X-ray v3 (Preprocessed + Filtered)",
        "epochs": 100,
        "batch_size": 16,
        "image_size": 1024,
        "patience": 10,
        "optimizer": "AdamW",
        "learning_rate": 0.001,
        "preprocessing": "grayscale + histogram_eq + normalization (NO blur)",
        "augmentation": "gaussian_blur (custom callback) + rotation (YOLO degrees=5.0)",
        "training_strategy": "minimal augmentation to preserve medical features",
        "gaussian_blur": "80% 3x3, 20% 5x5 with sigma=0.5",
    }
)

print("✓ WandB initialized successfully")
print(f"  Project: chest-xray-abnormality-detection")
print(f"  Run name: {wandb.run.name}")
print(f"  Run URL: {wandb.run.url}")

✓ WandB initialized successfully
  Project: chest-xray-abnormality-detection
  Run name: yolov11s-gaussian-blur-rotation
  Run URL: https://wandb.ai/minhquana-university-of-transportation-and-communication/chest-xray-abnormality-detection/runs/mj65kcd1


In [6]:
# Enable WandB integration in Ultralytics
settings.update({'wandb': True})

print("✓ WandB integration enabled for Ultralytics YOLO")
print("\nTraining metrics will be automatically logged to WandB:")
print("   - Loss curves (box_loss, cls_loss, dfl_loss)")
print("   - mAP scores (mAP50, mAP50-95)")
print("   - Learning rate schedules")
print("   - Training/validation images with predictions")

✓ WandB integration enabled for Ultralytics YOLO

Training metrics will be automatically logged to WandB:
   - Loss curves (box_loss, cls_loss, dfl_loss)
   - mAP scores (mAP50, mAP50-95)
   - Learning rate schedules
   - Training/validation images with predictions


## Section 4: Training Configuration

Configure training parameters with minimal augmentation strategy.

In [7]:
# Training configuration
training_config = {
    # Data
    'data': str(data_yaml),
    
    # Training hyperparameters
    'epochs': 100,
    'batch': 16,
    'imgsz': 1024,
    'patience': 10,
    'save': True,
    'plots': True,
    'verbose': True,
    
    # Device and performance
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'workers': 8,
    'cache': False,
    
    # Optimization parameters
    'optimizer': 'AdamW',
    'lr0': 0.001,           # Initial learning rate
    'lrf': 0.0001,          # Final learning rate (lr0 * lrf)
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'cos_lr': True,         # Use cosine learning rate scheduler
    
    # Only rotation is enabled from YOLO built-in augmentations
    'degrees': 5.0,
    'hsv_h': 0.0,    
    'hsv_s': 0.0,  
    'hsv_v': 0.0,    
    'translate': 0.0,   
    'scale': 0.0,    
    'shear': 0.0,          
    'perspective': 0.0,  
    'fliplr': 0.0, 
    'flipud': 0.0,
    'mosaic': 0.0,
    'mixup': 0.0,  
    'copy_paste': 0.0,    
    'auto_augment': None,  
    'erasing': 0.0,        
}

print("Training Configuration")
print("=" * 80)
for key, value in training_config.items():
    print(f"  {key:25s}: {value}")
print("=" * 80)

Training Configuration
  data                     : data/preprocessed_with_aug/data.yaml
  epochs                   : 100
  batch                    : 16
  imgsz                    : 1024
  patience                 : 10
  save                     : True
  plots                    : True
  verbose                  : True
  device                   : cuda
  workers                  : 8
  cache                    : False
  optimizer                : AdamW
  lr0                      : 0.001
  lrf                      : 0.0001
  momentum                 : 0.937
  weight_decay             : 0.0005
  warmup_epochs            : 3.0
  warmup_momentum          : 0.8
  warmup_bias_lr           : 0.1
  cos_lr                   : True
  degrees                  : 5.0
  hsv_h                    : 0.0
  hsv_s                    : 0.0
  hsv_v                    : 0.0
  translate                : 0.0
  scale                    : 0.0
  shear                    : 0.0
  perspective              : 0.0
  fl

## Section 5: Model Training

Train YOLOv11s with custom augmentation callback.

In [8]:
# Load YOLOv11s model
print("\nLoading YOLOv11s model...")
model = YOLO('backend/models/yolo11s.pt')

print("✓ Model loaded successfully")
print(f"  Model architecture: YOLOv11s")
print(f"  Parameters: ~{sum(p.numel() for p in model.model.parameters()) / 1e6:.1f}M")

print("\nStarting training...")
print("Progress will be tracked in WandB dashboard")
print("-" * 80)


Loading YOLOv11s model...
✓ Model loaded successfully
  Model architecture: YOLOv11s
  Parameters: ~9.5M

Starting training...
Progress will be tracked in WandB dashboard
--------------------------------------------------------------------------------


In [ ]:
# Train the model
try:
    results = model.train(
        **training_config,
        project='chest-xray-training',
        name='yolov11s-gaussian-blur-rotation'
    )
    
    print("\n" + "=" * 80)
    print("✓ Training completed successfully!")
    print("=" * 80)
    
    # Display results
    print("\nTraining Results:")
    if hasattr(results, 'results_dict'):
        print(f"  Best mAP50: {results.results_dict.get('metrics/mAP50(B)', 'N/A')}")
        print(f"  Best mAP50-95: {results.results_dict.get('metrics/mAP50-95(B)', 'N/A')}")
    
    # Save best model path
    best_model_path = Path(results.save_dir) / 'weights' / 'best.pt'
    print(f"\nBest model saved to: {best_model_path}")
    
except Exception as e:
    print(f"\nTraining failed: {e}")
    import traceback
    traceback.print_exc()
    raise

New https://pypi.org/project/ultralytics/8.3.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.226 🚀 Python-3.12.3 torch-2.9.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3060, 11906MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=None, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data/preprocessed_with_aug/data.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.0, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.0001, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=backend/models/yolo11s.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=yolov11

## Section 6: Model Validation

In [ ]:
# Validate on test set
print("Model Validation on Test Set")
print("=" * 80)

# Load best model
if 'best_model_path' in locals() and best_model_path.exists():
    print(f"Loading best model: {best_model_path}")
    model = YOLO(str(best_model_path))
else:
    print("Using last trained model")

print("\nRunning validation...")
metrics = model.val(data=str(data_yaml), split='test')

print("\nValidation Results:")
print("=" * 80)
results_dict = metrics.results_dict
print(f"  mAP50:       {results_dict.get('metrics/mAP50(B)', 0):.4f}")
print(f"  mAP50-95:    {results_dict.get('metrics/mAP50-95(B)', 0):.4f}")
print(f"  Precision:   {results_dict.get('metrics/precision(B)', 0):.4f}")
print(f"  Recall:      {results_dict.get('metrics/recall(B)', 0):.4f}")
print("=" * 80) 

## Section 7: Model Export

In [ ]:
# Export to backend
backend_models_dir = Path('backend/models')
backend_models_dir.mkdir(parents=True, exist_ok=True)

target_model_path = backend_models_dir / 'yolov11s_finetuned.pt'

print("Exporting Model to Backend")
print("=" * 80)

if 'best_model_path' in locals() and best_model_path.exists():
    print(f"Source: {best_model_path}")
    print(f"Target: {target_model_path}")
    
    shutil.copy(best_model_path, target_model_path)
    
    if target_model_path.exists():
        size_mb = target_model_path.stat().st_size / (1024*1024)
        print(f"\n✓ Model exported successfully!")
        print(f"  File size: {size_mb:.2f} MB")
        print(f"  Location: {target_model_path}")
        print(f"\nModel ready for production use!")
    else:
        print("Export failed")
else:
    print("Best model not found - cannot export")

print("=" * 80)

In [ ]:
# Close WandB run
wandb.finish()
print("✓ WandB run finished")

## Section 8: Training Summary

In [ ]:
print("\nTRAINING SUMMARY")
print("=" * 80)

print("\nCompleted Tasks:")
print("  1. ✓ Loaded preprocessed data from data/preprocessed/")
print("  2. ✓ Applied custom Gaussian blur augmentation during training")
print("  3. ✓ Applied YOLO rotation augmentation (±5°)")
print("  4. ✓ Trained YOLOv11s model for 100 epochs with early stopping")
print("  5. ✓ Tracked training with WandB")
print("  6. ✓ Validated on test set")
print("  7. ✓ Exported best model to backend/models/")

print("\nFinal Metrics:")
if 'results_dict' in locals():
    print(f"  mAP50:       {results_dict.get('metrics/mAP50(B)', 'N/A')}")
    print(f"  mAP50-95:    {results_dict.get('metrics/mAP50-95(B)', 'N/A')}")
    print(f"  Precision:   {results_dict.get('metrics/precision(B)', 'N/A')}")
    print(f"  Recall:      {results_dict.get('metrics/recall(B)', 'N/A')}")

print("\nNext Steps:")
print("  1. Run compare_baseline_finetuned.ipynb to compare with baseline")
print("  2. Check WandB dashboard for detailed training metrics")
print("  3. Test model in production via backend API")

print("\n" + "=" * 80)